In [54]:
# from google.colab import drive
# drive.mount('/content/drive')

In [55]:
# !pip install tabpfn
# !pip install -U pytorch_tabular
# !pip install pytorch-tabular
# !pip install --upgrade pytorch-tabular

In [56]:
import warnings
warnings.filterwarnings('ignore')

In [57]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
    CategoryEmbeddingModelConfig,
    FTTransformerConfig,
    TabNetModelConfig,
    GatedAdditiveTreeEnsembleConfig,
    TabTransformerConfig,
    AutoIntConfig
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [58]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

In [59]:
import pandas as pd
import numpy as np
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.tab_transformer import TabTransformerConfig
import pickle
from sklearn.metrics import mean_squared_error

# Đọc dữ liệu từ các file CSV riêng biệt
train = pd.read_csv('E:/IAD/Intern/05_CMAPSS/CMAPSSData/train_FD004.txt', sep=' ', header=None)
test = pd.read_csv('E:/IAD/Intern/05_CMAPSS/CMAPSSData/test_FD004.txt', sep=' ', header=None)

In [60]:
# Xóa các cột trống (nếu có)
train.dropna(axis=1, how='all', inplace=True)
test.dropna(axis=1, how='all', inplace=True)
# Lấy số lượng cột hiện tại của DataFrame
num_columns_train = len(train.columns)
num_columns_test = len(test.columns)

In [61]:
# Tạo tên cột tự động dựa trên số lượng cột
# cột đầu tiên là 'unit', cột thứ hai là 'cycle', 3 cột tiếp theo là 'setting1', 'setting2', 'setting3'
column_names_train = ['unit', 'cycle'] + [f'setting{i}' for i in range(1, 4)] + [f'sensor{i}' for i in range(1, num_columns_train - 5 + 1)]
column_names_test = ['unit', 'cycle'] + [f'setting{i}' for i in range(1, 4)] + [f'sensor{i}' for i in range(1, num_columns_test - 5 + 1)]

In [62]:
# Đặt tên cho các cột
train.columns = column_names_train
test.columns = column_names_test

In [63]:
# Tính toán RUL cho tập train
max_cycle_train = train.groupby('unit')['cycle'].max()
train['RUL'] = train.apply(lambda row: max_cycle_train[row['unit']] - row['cycle'], axis=1)

# Tính toán RUL cho tập test
max_cycle_test = test.groupby('unit')['cycle'].max()
test['RUL'] = test.apply(lambda row: max_cycle_test[row['unit']] - row['cycle'], axis=1)

In [64]:
train_copy = train.copy()
test_copy = test.copy()
#train = train_copy
#test = test_copy
#train = train.drop(columns=['unit', 'cycle'])
#test = test.drop(columns=['unit', 'cycle'])



In [65]:
from sklearn.preprocessing import OrdinalEncoder
# Xác định các cột đặc trưng (features) và cột mục tiêu (target)
features = column_names_train[2:]  # All columns except 'unit' and 'cycle'
target = ['RUL']

# Mã hóa các cột phân loại (Ordinal Encoding) cho tập train và test
cat_cols = ['unit']
train_enc = train.drop(columns=['cycle'])
test_enc = test.drop(columns=['cycle'])

# train_enc = train.copy()
# test_enc = test.copy()

# for col in cat_cols:
#     enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)
#     train_enc[col] = enc.fit_transform(train_enc[col].values.reshape(-1,1))
#     test_enc[col] = enc.transform(test_enc[col].values.reshape(-1,1))


# Cấu hình cho TabularModel
data_config = DataConfig(
    target=target,
    continuous_cols=features,  # Columns that are continuous features
    categorical_cols=['unit'], # Columns that are categorical
)

In [66]:
# Cấu hình cho TabTransformer
model_config = TabTransformerConfig(
    task="regression",
    learning_rate=1e-3,
    loss="MSELoss",
    metrics=["mean_squared_error"],
    metrics_params=[{}]
    #metrics_params=[{"mean_squared_error":{}}],
)

# Cấu hình huấn luyện
trainer_config = TrainerConfig(
    max_epochs=10, # Số lượng epoch để huấn luyện
    #gpus=1, # Sử dụng GPU nếu có
)

optimizer_config = OptimizerConfig()

In [67]:
# Khởi tạo mô hình
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    
)

2024-09-01 10:33:45,642 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

In [68]:
train_enc.head()

,unit,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL
0,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,...,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,320.0
1,1,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,...,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,319.0
2,1,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,...,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,318.0
3,1,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,...,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,317.0
4,1,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,9.00,...,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,316.0


In [69]:
rul_train = train_enc.loc[:, 'RUL']
rul_train.columns = ['RUL']
train_enc = train_enc.drop(columns=['RUL'])
train_enc.head()

,unit,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,9.00,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


In [70]:
rul_test = test_enc.loc[:, 'RUL']
test_enc = test_enc.drop(columns=['RUL'])
test_enc.head()

,unit,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,20.0072,0.7000,100.0,491.19,606.67,1481.04,1227.81,9.35,13.60,...,313.03,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007
1,1,24.9984,0.6200,60.0,462.54,536.22,1256.17,1031.48,7.05,9.00,...,163.61,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748
2,1,42.0000,0.8420,100.0,445.00,549.23,1340.13,1105.88,3.91,5.69,...,129.98,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365
3,1,42.0035,0.8402,100.0,445.00,549.19,1339.70,1107.26,3.91,5.69,...,129.48,2387.90,8078.89,9.3594,0.02,328,2212,100.00,10.56,6.2367
4,1,35.0079,0.8400,100.0,449.44,555.10,1353.04,1117.80,5.48,7.97,...,181.82,2387.87,8057.83,9.3030,0.02,333,2223,100.00,14.85,8.9326


In [72]:
train.head()

,unit,cycle,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,320.0
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,319.0
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,318.0
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,317.0
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,316.0


In [73]:
# Huấn luyện mô hình

tabular_model.fit(train)

Seed set to 42


2024-09-01 10:35:56,248 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-09-01 10:35:56,272 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-09-01 10:35:56,398 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: TabTransformerModel

2024-09-01 10:35:56,445 - {pytorch_tabular.tabular_model:340} - INFO - Preparing the Trainer

...

Trainable params: 279 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 279 K                                                                                                
Total estimated model params size (MB): 1

Output()

`Trainer.fit` stopped: `max_epochs=10` reached.


2024-09-01 10:48:09,432 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-09-01 10:48:09,450 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

In [ ]:
# Đánh giá mô hình
tabular_model.evaluate(test)

In [ ]:
# Dự đoán trên tập test
test_pred = tabular_model.predict(test)

In [ ]:
# Tính toán và in lỗi MSE
mse = mean_squared_error(test[target], test_pred["RUL_prediction"].values)
print(f"Mean Squared Error: {mse}")

# Lưu mô hình đã huấn luyện
with open('E:/IAD/Intern/05_CMAPSS/source/tab_transformer_cmapss.sav', 'wb') as model_file:
    pickle.dump(tabular_model, model_file)

# Tải mô hình đã lưu (nếu cần thiết)
with open('E:/IAD/Intern/05_CMAPSS/source/tab_transformer_cmapss.sav', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

In [ ]:
#test_enc = train.drop(columns=['unit','cycle','RUL'])
train_enc = train.drop(columns=['unit','cycle','RUL'])
test_enc = test.drop(columns=['unit','cycle','RUL'])
print("Columns in test_enc:", test_enc.columns.tolist())
print("Expected categorical columns:", cat_cols)
print("Expected feature columns:", features)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt


test_pred = loaded_model.predict(test_enc)
#mse = mean_squared_error(test[target], test_pred["RUL_prediction"].values)
#print(f"Mean Squared Error: {mse}")

mse = mean_squared_error(test_enc[target[0]], test_pred)
#rmse = np.sqrt(mse)
#mae = mean_absolute_error(test[target[0]], test_pred["RUL_prediction"].values)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

# Giả sử test_pred là một DataFrame, chuyển nó thành mảng numpy và làm phẳng
test_pred_flat = test_pred.to_numpy().ravel()


comparison_df = pd.DataFrame({
    'Actual RUL': test_enc[target[0]].values,
    'Predicted RUL': test_pred
})


print(comparison_df.head(10))

plt.figure(figsize=(10, 6))
plt.scatter(range(len(test_enc)), test_enc[target[0]], label='Actual', alpha=0.6)
plt.scatter(range(len(test_enc)), test_pred, label='Predicted', alpha=0.6)
plt.title('Actual vs Predicted RUL')
plt.xlabel('Sample Index')
plt.ylabel('RUL')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
errors = test_enc[target[0]].values - test_pred
plt.hist(errors, bins=50, edgecolor='k', alpha=0.7)
plt.title('Distribution of Prediction Errors')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.show()
# Tạo DataFrame với các mảng đã được làm phẳng
# comparison_df = pd.DataFrame({
#     'Actual RUL': test[target[0]].values.ravel(),  # Dùng .ravel() để làm phẳng mảng
#     'Predicted RUL': test_pred_flat  # Sử dụng mảng đã được làm phẳng
# })

# print(comparison_df.head(10))

# plt.figure(figsize=(10, 6))
# plt.scatter(range(len(test)), test[target[0]], label='Actual', alpha=0.6)
# plt.scatter(range(len(test)), test_pred, label='Predicted', alpha=0.6)
# plt.title('Actual vs Predicted RUL')
# plt.xlabel('Sample Index')
# plt.ylabel('RUL')
# plt.legend()
# plt.show()

# plt.figure(figsize=(10, 6))
# errors = test[target[0]].values - test_pred["RUL_prediction"]
# plt.hist(errors, bins=50, edgecolor='k', alpha=0.7)
# plt.title('Distribution of Prediction Errors')
# plt.xlabel('Prediction Error')
# plt.ylabel('Frequency')
# plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#max_cycles = df_spark.groupby('unit')['cycle'].max()
#df_spark['RUL'] = df_spark.apply(lambda row: max_cycles[row['unit']] - row['cycle'], axis=1)

# Tính toán RUL cho tập train
max_cycle_train = train.groupby('unit')['cycle'].max()
train['RUL'] = train.apply(lambda row: max_cycle_train[row['unit']] - row['cycle'], axis=1)

# Tính toán RUL cho tập test (nếu cần thiết)
max_cycle_test = test.groupby('unit')['cycle'].max()
test['RUL'] = test.apply(lambda row: max_cycle_test[row['unit']] - row['cycle'], axis=1)

#train, test = train_test_split(df_spark, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train.iloc[:, 2:25])
test_scaled = scaler.transform(test.iloc[:, 2:25])

X_train, y_train = train_scaled, train['RUL']
X_test, y_test = test_scaled, test['RUL']

In [ ]:
!pip install shap

import shap

#explainer = shap.Explainer(tabular_model, X_train)

explainer = shap.Explainer(predict_func, X_train)
#shap_values = explainer(X_test, check_additivity=False)

#shap_values = explainer(X_test, check_additivity=False)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test, feature_names=column_names[2:25])